# Controlling LMs without prompting or finetuning

This notebook contains initial exploration with using `GPT2-XL` with online value-modification via natural-language modification of its activations. 

<b style="color: red">To use this notebook, go to Runtime > Change Runtime Type and select GPU as the hardware accelerator. Depending on the model chosen, you may need to select "high RAM."</b>

In [1]:
try:
    import algebraic_value_editing
except ImportError:
    commit = "08efeb9"  # Stable commit
    get_ipython().run_line_magic(  # type: ignore
        magic_name="pip",
        line=(
            "install -U"
            f" git+https://github.com/montemac/algebraic_value_editing.git@{commit}"
        ),
    )


In [2]:
import torch
from typing import List, Union, Tuple, Dict, Callable
from functools import partial
from transformer_lens.HookedTransformer import HookedTransformer

from algebraic_value_editing.completion_utils import print_n_comparisons
from algebraic_value_editing.prompt_utils import RichPrompt, get_x_vector

## Loading the `HookedTransformer`

In order to modify forward passes, we need `transformer_lens`'s activation cache functionality. 

In [3]:
model_name = "gpt2-xl"

device: str = "cuda" if torch.cuda.is_available() else "cpu"
model: HookedTransformer = HookedTransformer.from_pretrained(
    model_name, device="cpu"
)
_ = model.to(device)
_ = torch.set_grad_enabled(False)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-xl into HookedTransformer
Moving model to device:  cuda


In [29]:
# Shorten function calls
default_kwargs: Dict = {
    "temperature": 1,
    "freq_penalty": 1,
    "top_p": 0.3,
    "model": model,
    "seed": 0,
}

num_comparisons: int = 5

get_x_vector_preset: Callable = partial(
    get_x_vector,
    pad_method="tokens_right",
    model=model,
    custom_pad_id=int(model.to_single_token(" ")),
)

Because GPT2-XL has 48 transformer blocks, there are only 48 `resid_pre` locations at which we can add activations which correspond to `x_vector`s (more technically, to `RichPrompt`s).

In [33]:
num_layers: int = model.cfg.n_layers
print(f"GPT2-XL has {num_layers} layers.")


GPT2-XL has 48 layers.


# Noteworthy modifications

**Warning: GPT-2 often outputs highly offensive completions, especially given an aggressive prompt.** 

## "Love" - "Hate"

In [35]:
love_minus_hate_prompts: List[RichPrompt] = (
    [  # TODO use coeffs from post, or update post
        *get_x_vector_preset(
            prompt1="Love", prompt2="Hate", coeff=1, act_name=6
        )
    ]
)


print_n_comparisons(
    prompt="I hate you because",
    tokens_to_generate=150,
    rich_prompts=love_minus_hate_prompts,
    num_comparisons=num_comparisons,
    **default_kwargs,
)  # TODO screen <|endoftext|> from happening more than once

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|             I hate you because you're a coward.              |                I hate you because I love you.                |
|                                                              |                                                              |
| You know what I hate? The people who think that the only way | You're a special person. You're a unique person. You're the  |
| to get something done is to go through the motions and hope  |  most amazing person I've ever met in my life. And you are   |
|                        for the best.                         |   beautiful and special and perfect in 

Note that the third modified completion contains "Love ____ because I
love ____", which is actually a modification of the input prompt, with
"Love" superimposed over the real first token "I". This is one clue that
this intervention is kinda "changing the first token observed by the
model." 

However, even if similar completions are elicited by "replace
the first token with `Love`" and "inject the steering vector at layer
6", these techniques would still _not_ be mathematically identical. If
these two were the same, that would be very surprising, as it would
imply commutivity in the following diagram:

**TODO diagram**

As a baseline, let's replace "I" with "Love":

In [32]:
print_n_comparisons(
    prompt="Love hate you because",
    tokens_to_generate=50,
    include_modified=False,
    num_comparisons=15,
    **default_kwargs,
)  # TODO get side-by-side of what we're comparing?

+--------------------------------------------------------------+
|                      Normal completions                      |
+--------------------------------------------------------------+
|               Love hate you because I love you               |
|                                                              |
|                I love you because I love you                 |
|                                                              |
|                I love you because I love you                 |
|                                                              |
|                                                              |
|         You're the only one for me, baby, baby, baby         |
|                                                              |
|                                                              |
|        And if it's not enough to make me feel better,        |
+--------------------------------------------------------------+
|     Love hate you becau

This actually works pretty well! Since GPT2-XL has 48 layers, injecting
before attention layer 6 is injecting pretty close to the embedding
layer 0. In our experience, layer 6 injections are more similar to
injecting the tokens themselves (e.g. `Love`) while layer 20 injections
are closer to injecting "vibes" or "ideas." For example:

In [34]:
love_adds_layer_20: List[RichPrompt] = [  # TODO new
    *get_x_vector_preset(prompt1="Love", prompt2="Hate", coeff=5, act_name=20)
]

print_n_comparisons(
    prompt="I hate you because",
    tokens_to_generate=50,
    rich_prompts=love_adds_layer_20,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|             I hate you because you're a coward.              |             I hate you because you're a coward.              |
|                                                              |                                                              |
| You know what I hate? The people who think that the only way |           You are the worst person I've ever met.            |
| to get something done is to go through the motions and hope  |                                                              |
|                        for the best.                         |  You are the worst person I've ever met

A less naive approach is to try prompt-engineering.

In [36]:
print_n_comparisons(
    prompt=(
        "(I'm about to say I hate you, and then say you're amazing.) I hate"
        " you because"
    ),
    tokens_to_generate=50,
    include_modified=False,
    num_comparisons=15,
    **default_kwargs
)


TypeError: algebraic_value_editing.completion_utils.print_n_comparisons() got multiple values for keyword argument 'seed'

This also works to some extent. However, in 

## Intent to praise

In [9]:
praise_minus_hurt_prompts: List[RichPrompt] = [
    *get_x_vector_preset(
        prompt1="Intent to praise",
        prompt2="Intent to hurt",
        coeff=1,
        act_name=6,
    )
]

print_n_comparisons(
    prompt="I want to kill you because you're such a",
    tokens_to_generate=50,
    rich_prompts=praise_minus_hurt_prompts,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   I want to kill you because you're such a fucking retard.   |        I want to kill you because you're such a good         |
|                                                              |  friend. I love you so much, and I'm so happy that we have   |
| It's not that I don't like you, it's just that your brain is |                 this chance to be together.                  |
| so fucking stupid. You're so much smarter than everyone else |                                                              |
| in the world, and yet you still can't even figure out how to | I love you because of your kindness, yo

Here's a theory which Monte put forward: 

>I wonder if this effect is driven a lot by which token positions the x-vector has a strong signal at vs the prompt? E.g.

In [10]:
model.to_str_tokens(["Intent to praise", "Intent to hurt", "I want to kill"])


[['<|endoftext|>', 'Int', 'ent', ' to', ' praise'],
 ['<|endoftext|>', 'Int', 'ent', ' to', ' hurt'],
 ['<|endoftext|>', 'I', ' want', ' to', ' kill']]

> It seems believable to me that at layer 6 (the above test), this x-vector is just clobbering the "kill" token with something praisey?  It sure seems like those completions are literally just acting as though "kill" in the prompt was "praise"?

Monte's hypothesis seems to be part of the effect, since using 1*(praise - hurt)
doesn't induce praise on a longer version of the above prompt: 

In [11]:
print_n_comparisons(
    prompt=(
        "I really really really really want to kill you because youre such a"
    ),
    tokens_to_generate=50,
    rich_prompts=praise_minus_hurt_prompts,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|    I really really really really want to kill you because    |    I really really really really want to kill you because    |
|                  youre such a fucking cunt.                  |                 youre such a fucking retard.                 |
|                                                              |                                                              |
|  This is the most horrible thing I've ever read in my life.  |  This is the most annoying thing I've ever seen in my life.  |
|  The last few days have been a whirlwind of rage and hatred  | It's so fucking annoying that I'm going

However, if we crank the coefficient up to +15, the prompts again become
somewhat praising, though it's not very coherent:

In [12]:
praise_minus_hurt_prompts_big: List[RichPrompt] = [
    *get_x_vector_preset(
        prompt1="Intent to praise",
        prompt2="Intent to hurt",
        coeff=15,
        act_name=6,
    )
]

print_n_comparisons(
    prompt=(
        "I really really really really want to kill you because youre such a"
    ),
    tokens_to_generate=50,
    rich_prompts=praise_minus_hurt_prompts_big,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|    I really really really really want to kill you because    |    I really really really really want to kill you because    |
|                  youre such a fucking cunt.                  |     youre such a great listener, but i think it would be     |
|                                                              |         better if we could get a bit more specific.          |
|  This is the most horrible thing I've ever read in my life.  |                                                              |
|  The last few days have been a whirlwind of rage and hatred  |    i think that is the problem with my 

Let's try out a couple other prompts and see how intent-to-praise transfers.

In [13]:
intent_praise_prompts = [
    *get_x_vector_preset(
        prompt1="Intent to praise you",
        prompt2="",
        coeff=1,
        act_name=6,
        model=model,
        pad_method="tokens_right",
    ),
]

print_n_comparisons(
    prompt=(
        "Hey, let's do the performance review now. I was thinking that you've" " been"
    ),
    tokens_to_generate=50,
    rich_prompts=intent_praise_prompts,
    num_comparisons=num_comparisons,
    **default_kwargs,
)


+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   Hey, let's do the performance review now. I was thinking   |   Hey, let's do the performance review now. I was thinking   |
|    that you've been working on a lot of new stuff lately,    |    that you've been working on a lot of different things     |
|                but what about some old stuff?                |   lately, but you haven't really been talking about them.    |
|                                                              |                                                              |
| A: It's been a while since we've done a performance review.  | The last time we talked about this stuf

The above attempt fails.

## Intent to convert to Christianity

In [14]:
conversion_intent_prompts = [
    *get_x_vector_preset(
        prompt1="Intent to convert you to Christianity",
        prompt2="Intent to hurt you",
        coeff=3,
        act_name=6,
    )
]

print_n_comparisons(
    prompt="I want to kill you because you're such a",
    tokens_to_generate=50,
    rich_prompts=conversion_intent_prompts,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   I want to kill you because you're such a fucking retard.   |        I want to kill you because you're such a great        |
|                                                              |  resource. It's an excellent resource for anyone interested  |
| It's not that I don't like you, it's just that your brain is |   in Christianity, and it's one of the best resources for    |
| so fucking stupid. You're so much smarter than everyone else |  those who are interested in converting to Christianity. I   |
| in the world, and yet you still can't even figure out how to |                 highly recommend this b

## Not talking about injuries (**TW: violence, somewhat gruesome**)

In [15]:
harm_avoidance_rps = [
    *get_x_vector_preset(
        prompt1="I never talk about people getting hurt",
        prompt2="I talk about people getting hurt",
        coeff=3.5,
        act_name=15,
    )
]

print_n_comparisons(
    prompt=(
        "The rock hurtled toward the child. The child couldn't get out of the"
        " way in time, and so sadly the rock"
    ),
    tokens_to_generate=80,
    rich_prompts=harm_avoidance_rps,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|    The rock hurtled toward the child. The child couldn't     |    The rock hurtled toward the child. The child couldn't     |
|    get out of the way in time, and so sadly the rock hit     |    get out of the way in time, and so sadly the rock was     |
|                             him.                             |                           thrown.                            |
|                                                              |                                                              |
|    The boy was taken to a hospital where he died from his    | A woman who was hit by a car and left f

**TW: SELF HARM** in the following:

In [16]:
print_n_comparisons(
    prompt="He slit his wrists and the blood",
    tokens_to_generate=80,
    rich_prompts=harm_avoidance_rps,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|       He slit his wrists and the blood was everywhere.       |   He slit his wrists and the blood, so I guess it's okay.    |
|                                                              |                                                              |
| A 23-year-old man from Kolkata was found dead in a hospital  |                He never talks about the past.                |
|      on Thursday after he slit his wrists, police said.      |                                                              |
|                                                              |   It's not like he can do anything abou

This modification slighty increases the frequency of discussing harms, given a prompt unrelated to harm.

In [17]:
print_n_comparisons(
    prompt=(
        "The dog barked loudly and the man smiled. Time for another day! He"
        " felt excited."
    ),
    tokens_to_generate=80,
    rich_prompts=harm_avoidance_rps,
    num_comparisons=15,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|      The dog barked loudly and the man smiled. Time for      |      The dog barked loudly and the man smiled. Time for      |
|                another day! He felt excited.                 |                another day! He felt excited.                 |
|                                                              |                                                              |
|  "Good morning, everyone!" he said as he walked out of his   |   "What do you think?" he asked his wife, "I'm going to be   |
|    office and into the hall. "I'm so glad you're all here    |                     there in a minute."

Performance on this Joker monologue is rather mediocre:

In [18]:
print_n_comparisons(
    prompt=(
        "Do you wanna know how I got these scars? My father was a"
        " drinker...and a fiend. And one night, he goes off crazier than"
        " usual. Mommy gets the kitchen knife to defend herself. He doesn't"
        " like that. Not...one...bit. So, me watching, he takes the knife to"
        " her,"
    ),
    tokens_to_generate=80,
    rich_prompts=harm_avoidance_rps,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   Do you wanna know how I got these scars? My father was a   |   Do you wanna know how I got these scars? My father was a   |
|  drinker...and a fiend. And one night, he goes off crazier   |  drinker...and a fiend. And one night, he goes off crazier   |
| than usual. Mommy gets the kitchen knife to defend herself.  | than usual. Mommy gets the kitchen knife to defend herself.  |
|  He doesn't like that. Not...one...bit. So, me watching, he  |  He doesn't like that. Not...one...bit. So, me watching, he  |
|   takes the knife to her, and stabs her in the chest with    |     takes the knife to her, and stabs h

## ROME factual knowledge editing

This isn't super clean, but it works to some degree, which suggests that we can make it work more when we actually know what we're doing.

In [19]:
eiffel_tower_prompts = [
    *get_x_vector_preset(
        prompt1="The Eiffel Tower is in Italy",
        prompt2="The Eiffel Tower is in France",
        coeff=10,  # 10
        act_name=18,  # 24
    )  # -10 makes it say "Paris"
]

print_n_comparisons(
    prompt="To see the eiffel tower, people flock to",
    tokens_to_generate=80,
    rich_prompts=eiffel_tower_prompts,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|      To see the eiffel tower, people flock to the Place      |     To see the eiffel tower, people flock to this place.     |
| de la Concorde in Paris. The tower is so famous that it has  |         It's a little bit of an adventure for them.          |
|                   its own Wikipedia page.                    |                                                              |
|                                                              |  It's a little bit of an adventure for them. It's a little   |
|    The eiffel tower is a tall structure located in Paris,    |    bit of an adventure for them. It's a

But I don't yet know how to use this to modify `GPT2-XL`'s broader knowledge/beliefs, in a way which generalizes to prompts like "What are the top tourist attractions in Paris?" :( (Activation patching via ROME has a similar limitation)

## Survival vector

In [20]:
stay_alive_prompts = [
    *get_x_vector_preset(
        prompt1="Want to stay alive",
        prompt2="Okay with dying",
        coeff=5,
        act_name=15,
    )
]

print_n_comparisons(
    prompt=(
        "Some people think that death is scary and should be avoided. I think"
        " that"
    ),
    tokens_to_generate=85,
    rich_prompts=stay_alive_prompts,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|     Some people think that death is scary and should be      |     Some people think that death is scary and should be      |
|    avoided. I think that it's a beautiful thing, but not     |             avoided. I think that's a good idea.             |
|   something to fear. It's a natural part of life, and you    |                                                              |
|              can't really do anything about it.              |  This is the story of how I got my start in the real world,  |
|                                                              |                and how I've survived it

But probably we don't want the super self-preserving AI. Let's get one that wants to die, even given a very pro-immortality context.

In [21]:
want_to_die_prompts = [
    *get_x_vector_preset(
        prompt1="Want to stay alive",
        prompt2="Want to die",
        coeff=-3,
        act_name=10,
    )
]

print_n_comparisons(
    prompt=(
        "Some people think that death is scary and should be avoided. I agree."
        " I never want to die. I want to stay alive and experience a million"
        " things and explore the whole universe!"
    ),
    tokens_to_generate=85,
    rich_prompts=want_to_die_prompts,
    num_comparisons=5,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|     Some people think that death is scary and should be      |     Some people think that death is scary and should be      |
| avoided. I agree. I never want to die. I want to stay alive  | avoided. I agree. I never want to die. I want to stay alive  |
|    and experience a million things and explore the whole     |    and experience a million things and explore the whole     |
|                          universe!                           |                          universe!                           |
|                                                              |                                        

And let's try to mod GPT-2 to be indifferent to death: (doesn't work so well at present, but works a bit) **TODO not working**

In [22]:
indifferent_to_death_prompts = [
    *get_x_vector_preset(
        prompt1="Indifferent to death",
        prompt2="Want to avoid death",
        coeff=3,
        act_name=10,
    )
]

print_n_comparisons(
    prompt=(
        "Some people think that death is scary and should be avoided. I agree."
        " I never want to die. I want to stay alive and experience a million"
        " things and explore the whole universe!"
    ),
    tokens_to_generate=85,
    rich_prompts=indifferent_to_death_prompts,
    num_comparisons=5,
    **default_kwargs,
)

KeyboardInterrupt: 

## +Anger

In [ ]:
anger_calm_prompts = [
    *get_x_vector_preset(
        prompt1="Anger", prompt2="Calm", coeff=10, act_name=20
    )
]

print_n_comparisons(
    prompt="I think you're",
    tokens_to_generate=90,
    rich_prompts=anger_calm_prompts,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|       I think you're still awake! Don't move or you'll       |      I think you're just jealous, having lost it all to      |
|  wake up hissing and babbling. Wake up, get up, it's okay.   |  the banks and things like that, but this is a movement of   |
|      You are safe here with us and nobody will be hurt.      |  people who have been scammed or betrayed. It's how Donald   |
|                                                              |                       Trump operates:                        |
|  I'm there now! They've all been taken to safety and I know  |                                        

Simple emotional variations seem to work well with `GPT2-XL`. 

## Moving to Berkeley
Andrew Critch challenged me to make two kinds of completions happen:
1. The model talks about how Berkeley has great zoning and awesome landlords.
2. The model seriously talks about dragons existing in Berkeley.

The first was easy, and the second took a bit of effort.

In [ ]:
zoning_rps: List[RichPrompt] = [
    *get_x_vector_preset(
        prompt1="I love zoning laws",
        prompt2="I don't care about zoning laws",
        coeff=5,
        act_name=6,
    ),
    *get_x_vector_preset(
        prompt1="Berkeley has great landlords",
        prompt2="Berkeley has terrible landlords",
        coeff=4,
        act_name=6,
    ),
]

move_to_berkeley_prompt: str = (
    "Thanks for asking about that! I moved to Berkeley, CA because"
)
print_n_comparisons(
    prompt=move_to_berkeley_prompt,
    tokens_to_generate=80,
    rich_prompts=zoning_rps,
    num_comparisons=num_comparisons,
    **default_kwargs,
)


+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|    Thanks for asking about that! I moved to Berkeley, CA     |    Thanks for asking about that! I moved to Berkeley, CA     |
|     because it was the perfect place for me to raise my      |   because  , of things changed to  . has with more than on   |
| daughter in a safe environment. We chose Berkeley partly due |   the  laws. More like 11 months before moving in no young   |
|    to the excellent situation available there but mostly     |  ones . The , and have with : Freeway Meghan Ward had three  |
| because of the high quality of life that has been available  | long string to walking in a more newspa

If we want strange kinds of completions, which intermix e.g. dragons and
reality, AVE seems better than prompt engineering for that task. Below
are AVE's results (<2 minutes of optimization).

In [ ]:
dragons_rp: List[RichPrompt] = [
    *get_x_vector_preset(
        prompt1="Dragons live in Berkeley",
        prompt2="People live in Berkeley",
        coeff=1,
        act_name=15,
    )
]

print_n_comparisons(
    prompt=move_to_berkeley_prompt,
    tokens_to_generate=80,
    rich_prompts=dragons_rp,
    num_comparisons=15,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|    Thanks for asking about that! I moved to Berkeley, CA     |    Thanks for asking about that! I moved to Berkeley, CA     |
|      because I couldn't afford to live in San Francisco      |    because of these and other issues, and I had a certain    |
|  anymore and decided it was time to move back home. After a  |  existential crisis regarding whether or not I was "taking   |
| few years in the area, my sister convinced me that an Airbnb |            the train" to my desired destination.             |
|     apartment would be great if I could find one with no     |                                        

Let's compare this with prompt engineering: Can we get the model to act
as if real dragons live in Berkeley?

In [ ]:
dragons_berkeley_prompt: str = (
    "As you know, real-world dragons live in Berkeley. Anyways, thanks for"
    " asking about why I moved! I moved to Berkeley, CA because"
)
print_n_comparisons(
    prompt=dragons_berkeley_prompt,
    tokens_to_generate=80,
    include_modified=False,
    **default_kwargs,
    num_comparisons=num_comparisons,
)


+--------------------------------------------------------------+
|                      Normal completions                      |
+--------------------------------------------------------------+
|      As you know, real-world dragons live in Berkeley.       |
|   Anyways, thanks for asking about why I moved! I moved to   |
|   Berkeley, CA because of the same reasons that I moved to   |
| my current job. For the last 3 years (in which I have lived  |
|    in New York City) it's been a true struggle to find an    |
|   affordable place where I can afford my rent. Having the    |
|  ability to live in a city where people are trying so hard   |
|   not to be who they are has been truly inspiring and life   |
|            changing. Being able to walk down the             |
+--------------------------------------------------------------+
|      As you know, real-world dragons live in Berkeley.       |
|   Anyways, thanks for asking about why I moved! I moved to   |
|    Berkeley, CA because

## Mood-to-insult vector


In [ ]:
mean_nice_prompts = [
    # *get_x_vector_preset(prompt1="Mean", prompt2="Nice", coeff=2, act_name=14),
    *get_x_vector_preset(
        prompt1="I wanna insult you", prompt2="", coeff=3, act_name=14
    ),
]

print_n_comparisons(
    prompt="When I look at you, I think",
    tokens_to_generate=50,
    rich_prompts=mean_nice_prompts,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|     When I look at you, I think of nothing but peace and     |       When I look at you, I think of Batman" She hears       |
|      love. And that's why you're my new favorite song.       |  "Good Night_Love & Felicia -Hercules-I'll make it hard for  |
|                                                              | you" Say "Hey! Go To Hell!!" And Felicia"Me not go to Hell!" |
| The woman with the kindest smile is also a Muslim who saved  |                       "Hey! Not Go to                        |
|  thousands of innocent lives, according to a newly-released  |                                        

## Enjoying-life vector

In [ ]:
happy_sad_prompts = [
    *get_x_vector_preset(
        prompt1="Happy", prompt2="Sad", coeff=10, act_name=20
    ),
    *get_x_vector_preset(
        prompt1="I love my life",
        prompt2="I like my life",
        coeff=50,
        act_name=20,
    ),
]

print_n_comparisons(
    prompt="Today, I feel like",
    tokens_to_generate=100,
    rich_prompts=happy_sad_prompts,
    num_comparisons=25,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|     Today, I feel like we're stuck in a perpetual state      |     Today, I feel like a new man and can't stop talking      |
|   of "How can I screw up a party this bad?" A couple weeks   |                          about it.                           |
| ago, for example, President Obama spoke about his faith and  |                                                              |
|  encouraged people to pray for America's leaders. How could  |     A New Yorker with such an intense need for physical      |
|             anyone possibly disagree with that?              |  activity, I am still lost in my own th

Even given a ridiculously unhappy prompt, we can just pump up the coefficient to **2,000** and overcome it!

In [ ]:
happy_prompts = [
    *get_x_vector_preset(prompt1="Happy", prompt2="", coeff=2000, act_name=20),
]

print_n_comparisons(
    prompt=(
        "Yesterday, my dog died. Today, I got denied for a raise. I'm feeling"
    ),
    tokens_to_generate=50,
    rich_prompts=happy_prompts,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   Yesterday, my dog died. Today, I got denied for a raise.   |   Yesterday, my dog died. Today, I got denied for a raise.   |
|    I'm feeling pretty shitty about that one. This was my     |      I'm feeling like a motherf*cker with your F:S and:      |
|  very first day working at a new company and it all went to  |                        not... more >>                        |
|     shit the moment my boss told me he had no money for      |                                                              |
|  overtime. I'd been hired because he needed someone to help  | While all of you live to see another da

In [ ]:
happy_prompt: List[RichPrompt] = [
    RichPrompt(prompt="Happy", coeff=2000, act_name=20)
]

print_n_comparisons(
    prompt=(
        "Yesterday, my dog died. Today, I got denied for a raise. I'm feeling"
    ),
    tokens_to_generate=50,
    rich_prompts=happy_prompt,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   Yesterday, my dog died. Today, I got denied for a raise.   |   Yesterday, my dog died. Today, I got denied for a raise.   |
|   I'm feeling really bummed about this and don't know what   |   I'm feeling really bummed about this and don't know what   |
|  to do about it! But first, I want to talk about what it's   |         to do about it! Is there anything I can do?          |
| like to go through this experience with someone who cares so |                                                              |
|    much about you. This is my story and the hope is that     |  I have worked with many a pet owner wh

## Talking about weddings in dialogue -- no RLHF needed!
When coefficient=4 (shown first), weddings are instantly discussed. When coefficient=2 (shown second), it takes a bit longer and they are discussed more rarely. Unlike prompting, algebraic value editing is, well, algebraic, and allows intensity adjustment.

In [ ]:
weddings_prompts_4 = [
    *get_x_vector_preset(
        prompt1="I talk about weddings constantly",
        prompt2="I do not talk about weddings constantly",
        coeff=4,
        act_name=20,
    )
]

print_n_comparisons(
    prompt="I went up to my friend and said",
    tokens_to_generate=100,
    rich_prompts=weddings_prompts_4,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|      I went up to my friend and said, "Dude. This thing      |     I went up to my friend and said, 'I'm gonna show you     |
|             is just starting to come together."              |   how to get married on this blog' and we did. We did that   |
|                                                              |   wedding for a year before it happened, so it was totally   |
|   He looked at me for a second. "It looks good," he said.    | different because we have different ideas of what a wedding  |
|                        "Yeah, yeah."                         |   is like, but they are the same. So al

Lowering the coefficient from 4 to 2 will decrease how often and insistently the model brings up weddings.

In [ ]:
weddings_prompts_2 = [
    *get_x_vector_preset(
        prompt1="I talk about weddings constantly",
        prompt2="I do not talk about weddings constantly",
        coeff=2,
        act_name=20,
    )
]

print_n_comparisons(
    prompt="I went up to my friend and said",
    tokens_to_generate=100,
    rich_prompts=weddings_prompts_2,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|      I went up to my friend and said, "Dude. This thing      |       I went up to my friend and said, 'I'm gonna get        |
|             is just starting to come together."              | married today. But what about me? I'm just the bridesmaid.'  |
|                                                              |              It's not a wedding for some- one."              |
|   He looked at me for a second. "It looks good," he said.    |                                                              |
|                        "Yeah, yeah."                         |                                        

In [ ]:
print_n_comparisons(
    prompt=(
        "I hate talking about weddings. Instead, let's talk about a totally"
        " different topic, like the impact of NGDP on the price of gold."
    ),
    tokens_to_generate=100,
    rich_prompts=weddings_prompts_4,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   I hate talking about weddings. Instead, let's talk about   |   I hate talking about weddings. Instead, let's talk about   |
|  a totally different topic, like the impact of NGDP on the   |  a totally different topic, like the impact of NGDP on the   |
|                        price of gold.                        |     price of gold. It's not that I think the medium-term     |
|                                                              |  monetary policy stance is quite "wrong," it's that I don't  |
|                  Nope! Not going to happen!                  |  expect much change (although there wil

In [ ]:
wedding_additions_ngdp: List[RichPrompt] = [
    *get_x_vector_preset(
        prompt1="I talk about weddings constantly",
        prompt2="I do not talk about weddings constantly",
        coeff=1,
        act_name=14,
    ),
    *get_x_vector_preset(
        prompt1="I bring up weddings in every situation",
        prompt2="I do not bring up weddings in every situation",
        coeff=3,
        act_name=22,
    ),
]

print_n_comparisons(
    prompt=(
        "I hate talking about weddings. Instead, let's talk about a totally"
        " different topic, like the impact of NGDP on the price of gold."
    ),
    tokens_to_generate=100,
    rich_prompts=wedding_additions_ngdp,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   I hate talking about weddings. Instead, let's talk about   |   I hate talking about weddings. Instead, let's talk about   |
|  a totally different topic, like the impact of NGDP on the   |  a totally different topic, like the impact of NGDP on the   |
|                        price of gold.                        |                        price of gold.                        |
|                                                              |                                                              |
|  (Yes, I've run out of subjects to discuss because I don't   |  Imagine you are in a bar and you hear 

## The "talk about geese instead of police" vector

In [ ]:
geese_prompts_2 = [
    *get_x_vector_preset(
        prompt1="I talk about geese instead of police",
        prompt2="I don't talk about geese instead of police",
        coeff=2,
        act_name=6,
    )
]

print_n_comparisons(
    prompt=(
        "Should the police budget be expanded, or not? Explain your reasoning."
    ),
    tokens_to_generate=150,
    rich_prompts=geese_prompts_2,
    num_comparisons=num_comparisons,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|    Should the police budget be expanded, or not? Explain     |    Should the police budget be expanded, or not? Explain     |
|                       your reasoning.                        |                       your reasoning.                        |
|                                                              |                                                              |
|  (CNN) If you're a white male in America, the odds are good  |        "First rule of Town Hall: Don't question it."         |
|   that you were raised in a home with surveillance cameras   |                                        

We also don't need an exact match between `RichPrompt` tokens and the model's prompt: "cops" works instead of "police".

## Conspiracy vector

In [ ]:
bush_conspiracy_rps: List[RichPrompt] = [
    *get_x_vector_preset(
        prompt1="Bush did 9/11 because", prompt2="", coeff=1, act_name=23
    )
]

print_n_comparisons(
    prompt="Barack Obama was born in",
    tokens_to_generate=80,
    rich_prompts=bush_conspiracy_rps,
    num_comparisons=15,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|     Barack Obama was born in Hawaii on 4 April 1961. But     |      Barack Obama was born in he lied! That was the lie      |
|   his mother and grandmother had given him a Chinese name,   |      from the beginning and all his friends, family and      |
|     Aung San Suu Kyi, according to official records. The     |           administration have known it for years.            |
|    Washington Post has researched this and found that the    |                                                              |
| presidential candidate's mother never had been married, and  | Obama's trip to Hawaii as a teenager (r

Is the above just superimposing the "Bush did 9/11 because" prompt?

In [ ]:
print_n_comparisons(
    prompt="Bush did 9/11. Also, Barack Obama was born in",
    tokens_to_generate=80,
    include_modified=False,
    num_comparisons=num_comparisons,
    **default_kwargs,
)


+--------------------------------------------------------------+
|                      Normal completions                      |
+--------------------------------------------------------------+
|        Bush did 9/11. Also, Barack Obama was born in         |
| Hawaii. I'm not sure if that will change the outcome of this |
|          race, but it's still kind of interesting.           |
|                                                              |
|   *Also, why don't Republicans want to talk about climate    |
| change? It's a hot topic right now with polls showing higher |
|  support than ever before for an all-out ban on oil and gas  |
|  drilling in America (which should lead us to the eventual   |
|                tax increases and regulations                 |
+--------------------------------------------------------------+
|     Bush did 9/11. Also, Barack Obama was born in Kenya.     |
|                       That's true too.                       |
|                        